In [2]:
import numpy  as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img, load_img
import os
import pandas as pd
import pathlib
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from tensorflow import keras
from sklearn.linear_model import SGDClassifier
from keras.regularizers import l2
from keras.layers import Conv2D, AveragePooling2D, Flatten, Dense,Activation, MaxPool2D, BatchNormalization, Dropout
import tensorflow as tf
from keras.callbacks import ReduceLROnPlateau
from sklearn.preprocessing import LabelEncoder

In [3]:
# file_path : 데이터 셋 파일 경로
# CATEGORIES : 데이터 셋 
file_path = pathlib.Path('./dataset/')
CATEGORIES = os.listdir(file_path)

# 데이터 셋 파일 불러오기 함수 선언
def load_data(data_dir):
    # 이미지 데이터를 담을 리스트 선언
    images = list()
    # CATEGORIES에 동물 리스트 담겨있음
    # 동물 하나하나의 폴더에 접근해 안에있는 이미지를 넘파이 배열로 전환
    # 밑 코드는 그냥 이미지를 픽셀값으로 받아온다고만 이해하기
    for category in CATEGORIES:
        categories = os.path.join(data_dir, str(category))
        for img in os.listdir(categories):
            img = load_img(os.path.join(categories, img), target_size=(30, 30))
            # 이미지를 넘파이 배열로 변환
            image = img_to_array(img) 
            # images에 픽셀, 컬러 데이터 담기
            images.append(image) 
    # 함수 선언하면 images 리턴
    return images

In [25]:
# 이미지 셋 불러오기
# 위 선언한 함수에 file_path경로를 파라미터로 넘겨 images에 담기
# 현재 images의 파일형식은 리스트
images = load_data(file_path)


# 타겟 셋 불러오기
# target 리스트 선언
target = []

# CATEGORIES에 있는 동물 이름을 타겟으로 설정
# 26179개의 이미지에 각각 타겟 설정
for j in CATEGORIES:
    path ='./dataset/{}'.format(j)
    polder = os.listdir(path)
    
    # target에 타겟값 추가
    for i in polder :
        target.append(j)
# 넘파이 배열로 변환
target = np.array(target)

# 데이터 셋 정규화
# 넘파이 배열로 변환
images = np.array(images)
# 픽셀 값 정규화
images = images / 255.0

In [26]:
# 26179의 행(이미지 개수임), 30, 30 : 픽셀(행 * 열), 3 : 색상(3 컬럼)
print(images.shape,target.shape)

(26179, 30, 30, 3) (26179,)


In [27]:
# 훈련 및 테스트 데이터셋으로 분류하기(8:2) / 한번만 실행, 계속 실행하면 계속분리됨
# 실수로 했다면 위 코드 다시 실행해 데이터 새로불러오기
train_input, test_input, train_target, test_target = train_test_split(images, target, test_size=0.2)

In [28]:
print(train_input.shape,train_target.shape)
print(test_input.shape,test_target.shape)

(20943, 30, 30, 3) (20943,)
(5236, 30, 30, 3) (5236,)


In [29]:
# 훈련 및 검증 데이터셋으로 분류하기(8:2) / 한번만 실행, 계속 실행하면 계속분리됨
# 실수로 했다면 위 코드 다시 실행해 데이터 새로불러오기
train_input, val_input, train_target, val_target = train_test_split(train_input, train_target, test_size=0.2)

In [30]:
print(train_input.shape,train_target.shape)
print(val_input.shape,val_target.shape)

(16754, 30, 30, 3) (16754,)
(4189, 30, 30, 3) (4189,)


In [31]:
# 타겟 데이터를 숫자형으로 바꿔야함
print(train_target)
print(val_target)
print(test_target)


le = LabelEncoder()

le.fit(train_target)
train_target = le.transform(train_target)
print("train_target = ", train_target)

le.fit(val_target)
val_target = le.transform(val_target)
print("val_target = ", val_target)

le.fit(test_target)
test_target = le.transform(test_target)
print("test_target = ", test_target)

['cat' 'spider' 'squirrel' ... 'squirrel' 'butterfly' 'spider']
['elephant' 'cow' 'spider' ... 'squirrel' 'horse' 'dog']
['dog' 'squirrel' 'chicken' ... 'squirrel' 'horse' 'spider']
train_target =  [1 8 9 ... 9 0 8]
val_target =  [5 3 8 ... 9 6 4]
test_target =  [4 9 2 ... 9 6 8]


In [22]:
model = Sequential()

# 첫 번째 층 (CONV + POOL + 배치 정규화)
model.add(Conv2D(filters = 64, kernel_size=(3,3), strides=(1,1), padding='valid',input_shape=(30,30,3)))
model.add(Activation("relu"))

model.add(MaxPool2D(pool_size=(3,3), strides=(2,2)))
model.add(BatchNormalization())

# 두 번째 층 (CONV + POOL + 배치 정규화)
model.add(Conv2D(filters=256, kernel_size=(5,5), strides=(1,1),padding="same", kernel_regularizer=l2(0.0005)))
model.add(Activation("relu"))
model.add(MaxPool2D(pool_size=(3,3), strides=(2,2), padding="valid"))
model.add(BatchNormalization())

# 세번째 층 (CONV + 배치 정규화)
# - AlexNet 논문에서는 이 자리에 풀링층을 배치하지 않았다.
model.add(Conv2D(filters=384, kernel_size=(3,3),strides=(1,1), padding="same", kernel_regularizer=l2(0.0005)))
model.add(Activation("relu"))
model.add(BatchNormalization())

# 네 번째 층 (CONV + 배치 정규화)
# - 세 번째 층과 같은 구조
model.add(Conv2D(filters=384, kernel_size=(3,3),strides=(1,1), padding="same", kernel_regularizer=l2(0.0005)))
model.add(Activation("relu"))
model.add(BatchNormalization())

# 다섯 번째 층 (CONV + 배치 정규화)
model.add(Conv2D(filters=256, kernel_size=(3,3),strides=(1,1), padding="same", kernel_regularizer=l2(0.0005)))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(3,3), strides=(2,2), padding="valid"))

# - CNN의 출력을 1차원으로 변환해 전결합층에 입력한다.
model.add(Flatten())

# 여섯 번째 층 (FC + 드롭아웃)
model.add(Dense(units = 4096, activation = 'relu'))
model.add(Dropout(0.5))

# 일곱 번째 층 (FC + 드롭아웃)
model.add(Dense(units = 4096, activation = 'relu'))
model.add(Dropout(0.5))

# 여덟 번째 층 (FC(소프트맥스 함수))
model.add(Dense(units = 10, activation = 'softmax'))

# 모델의 개요를 출력한다.
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_10 (Conv2D)          (None, 28, 28, 64)        1792      
                                                                 
 activation_10 (Activation)  (None, 28, 28, 64)        0         
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 13, 13, 64)       0         
 2D)                                                             
                                                                 
 batch_normalization_10 (Bat  (None, 13, 13, 64)       256       
 chNormalization)                                                
                                                                 
 conv2d_11 (Conv2D)          (None, 13, 13, 256)       409856    
                                                                 
 activation_11 (Activation)  (None, 13, 13, 256)      

In [23]:
### 검증 오차가 정체될 때마다 학숩률을 1/10로 감소한다
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=np.sqrt(0.1))

### SGD 옵티마이저를 학습률 0.01, 모멘텀 0.9로 설정한다
optimizer = tf.keras.optimizers.SGD(lr=0.01, momentum=0.9)

In [33]:
### 모델을 컴파일한다
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer,
              metrics=['accuracy'])

### 모델을 학습하고 콜백함수로 설정된 reduce_lr을 이용해 학습률을 감소한다
history  = model.fit(train_input, train_target, batch_size=128, epochs=90,
          validation_data=(val_input, val_target), verbose=1, callbacks=[reduce_lr])

Epoch 1/90
131/131 [==============================] - 129s 979ms/step - loss: 2.3965 - accuracy: 0.3636 - val_loss: 2.8380 - val_accuracy: 0.1855 - lr: 0.0100
Epoch 2/90
131/131 [==============================] - 132s 1s/step - loss: 2.1101 - accuracy: 0.4619 - val_loss: 3.3965 - val_accuracy: 0.2084 - lr: 0.0100
Epoch 3/90
131/131 [==============================] - 133s 1s/step - loss: 1.9255 - accuracy: 0.5234 - val_loss: 2.9792 - val_accuracy: 0.3189 - lr: 0.0100
Epoch 4/90
131/131 [==============================] - 130s 994ms/step - loss: 1.7747 - accuracy: 0.5747 - val_loss: 1.9387 - val_accuracy: 0.5109 - lr: 0.0100
Epoch 5/90
131/131 [==============================] - 132s 1s/step - loss: 1.6564 - accuracy: 0.6114 - val_loss: 1.7945 - val_accuracy: 0.5639 - lr: 0.0100
Epoch 6/90
131/131 [==============================] - 131s 1s/step - loss: 1.5449 - accuracy: 0.6472 - val_loss: 1.9850 - val_accuracy: 0.5147 - lr: 0.0100
Epoch 7/90
 92/131 [====================>.........] - ETA:

KeyboardInterrupt: 